In [68]:
import datetime
import pymongo
import random
import uuid
from pymongo import MongoClient

In [69]:
N_BOOKMARKS = 15000 # number of bookmarks
N_MOVIES_LIKES = 15000 

In [70]:
client = MongoClient('mongodb', 27017)

In [71]:
# Getting a Database
db = client.research_db

In [72]:
# List of bookmarks
# Scheme:
# document = 
# {user_id: 
#     [movie_id_1, movie_id_2]
# }
bookmarks = db.bookmarks

In [73]:
# Loading data

for i in range(N_BOOKMARKS):
    user_id = str(uuid.uuid4())
    movie_id = str(uuid.uuid4())
    bookmarks.insert_one(
        {"id": i,
         "user_id": user_id,
         "movie_id": [movie_id]
        }
    )


In [74]:
# Reading data
start = datetime.datetime.now()
for i in range(N_BOOKMARKS):
    id = random.randint(0, 15000)
    bookmarks.find_one({"id": id})
end = datetime.datetime.now()
print(end-start)

0:01:00.668124


In [75]:
# Likes - like is a score
# Scheme:
# documents = 
# [{movie_id, user_id, score}]
likes = db.likes

In [76]:
# Loading data

for i in range(N_MOVIES_LIKES):
    user_id = str(uuid.uuid4())
    movie_id = random.randint(0,N_MOVIES_LIKES)
    score = random.randint(0,10)
    likes.insert_one(
          {"movie_id": movie_id,
           "user_id": user_id,
           "score": score
          }
    )

In [77]:
# Reading data
start = datetime.datetime.now()
for i in range(N_MOVIES_LIKES):
    movie_id = random.randint(0,N_MOVIES_LIKES)
    pipeline = [{"$match":
                    {"movie_id": movie_id},  
                },
                {"$group":
                    {
                        "_id": "$movie_id",
                        "avgscore": {"$avg": "$score"},
                    },
                },
                ]
    for doc in likes.aggregate(pipeline):
        rating = doc["avgscore"]
end = datetime.datetime.now()
print(end-start)

0:06:14.844879


In [83]:
#Real-time
# Load data
deltas = []
pipeline = [{"$match":
                    {"movie_id": movie_id},  
                },
                {"$group":
                    {
                        "_id": "$movie_id",
                        "avgscore": {"$avg": "$score"},
                    },
                },
                ]
for i in range(10):
    #Load data
    user_id = str(uuid.uuid4())
    movie_id =  str(uuid.uuid4())
    score = random.randint(0,10)
    likes.insert_one(
              {"movie_id": movie_id,
               "user_id": user_id,
               "score": score
              }
        )
    
    # Read data 
    start = datetime.datetime.now()    
    for doc in likes.aggregate(pipeline):
        rating = doc["avgscore"]
    end = datetime.datetime.now()
    diff = end-start
    print(end-start)
    print(diff.microseconds)
    deltas.append(diff.microseconds)
print(sum(deltas)/10)

0:00:00.033729
33729
0:00:00.026347
26347
0:00:00.027767
27767
0:00:00.023854
23854
0:00:00.027181
27181
0:00:00.026651
26651
0:00:00.026431
26431
0:00:00.027377
27377
0:00:00.027701
27701
0:00:00.027832
27832
27487.0
